In [1]:
import pandas as pd
import numpy as nm
import glob

# Fuel Consumption Ratings
We will start by defining the column names.
> the column `fc_combined_mpg` is not ment to be kept, it is defined to facilitate droping it later
> using the `drop` function.

In [2]:
fc_df_columns = ['model_year', 'make', 'model_name', 'vehicle_class', 'engine_size', 'cylinders', 'transmission', 'fuel_type', 'fc_city', 'fc_highway', 'fc_combined','fc_combined_mpg', 'co2_emissions']

For each csv file, read the content into a dataframe and apply some cleaning, after that collect all the dataframes into a list.

In [3]:
fc_df_array = []

for file_name in glob.glob("../raw_data/fuel_consumption/MY*Ratings*.csv"):
    # skip the header rows
    df_temp = pd.read_csv(file_name, skiprows=[0, 1, 2])
    # drop all empty lines and columns
    df_temp.dropna(how='all', axis='columns', inplace=True)
    df_temp.dropna(how='all', axis='index', inplace=True)
    
    # remove embeded data-dictionary
    df_temp = df_temp.iloc[:-23]

    
    # from 2017 to 2022, there are two additional columns added
    # for the sake of having a bigger dataset, we'll remove
    # those two columns
    if df_temp.shape[1] > 13:
        df_temp = df_temp.iloc[:-23, :13]
    else:
         df_temp = df_temp.iloc[:-23, :]

    # provide column names
    df_temp.columns = fc_df_columns
    
    #drop uneeded columns
    df_temp.drop(labels="fc_combined_mpg", axis="columns", inplace=True)

    fc_df_array.append(df_temp)

C:\Users\dell\AppData\Local\Temp\ipykernel_10004\2172513345.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_name, skiprows=[0, 1, 2])
C:\Users\dell\AppData\Local\Temp\ipykernel_10004\2172513345.py:5: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_name, skiprows=[0, 1, 2])
C:\Users\dell\AppData\Local\Temp\ipykernel_10004\2172513345.py:5: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_name, skiprows=[0, 1, 2])
C:\Users\dell\AppData\Local\Temp\ipykernel_10004\2172513345.py:5: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_name, skiprows=[0, 1, 2])


Merge all the dataframes using the column names

In [4]:
fc_df = pd.concat(fc_df_array, ignore_index=True)

Some columns are not of the correct datatype, while for other columns we need to change the values into more readable ones.

In [5]:
fc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25866 entries, 0 to 25865
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   model_year     25866 non-null  object 
 1   make           25866 non-null  object 
 2   model_name     25866 non-null  object 
 3   vehicle_class  25866 non-null  object 
 4   engine_size    25866 non-null  float64
 5   cylinders      25866 non-null  float64
 6   transmission   25866 non-null  object 
 7   fuel_type      25866 non-null  object 
 8   fc_city        25866 non-null  float64
 9   fc_highway     25866 non-null  float64
 10  fc_combined    25866 non-null  float64
 11  co2_emissions  25866 non-null  float64
dtypes: float64(6), object(6)
memory usage: 2.4+ MB


In [6]:
fc_df = fc_df.astype({"model_year": nm.int64, 'cylinders': nm.int64, "co2_emissions": nm.int64})

In [7]:
fc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25866 entries, 0 to 25865
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   model_year     25866 non-null  int64  
 1   make           25866 non-null  object 
 2   model_name     25866 non-null  object 
 3   vehicle_class  25866 non-null  object 
 4   engine_size    25866 non-null  float64
 5   cylinders      25866 non-null  int64  
 6   transmission   25866 non-null  object 
 7   fuel_type      25866 non-null  object 
 8   fc_city        25866 non-null  float64
 9   fc_highway     25866 non-null  float64
 10  fc_combined    25866 non-null  float64
 11  co2_emissions  25866 non-null  int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 2.4+ MB


In [8]:
def fuel_type_name(short_name):
    long_name = None
    if short_name == "X":
        long_name = "regular gasoline"
    elif short_name == "Z":
        long_name = "premium gasoline"
    elif short_name == "D":
        long_name = "diesel"
    elif short_name == "E":
        long_name = "ethanol e85"
    elif short_name == "N":
        long_name = "natural gaz"
    else:
        raise ValueError("fuel type not recognized")
    
    return long_name

fc_df['fuel_type'] = fc_df['fuel_type'].map(fuel_type_name)

In [9]:
fc_df['fuel_type'].unique()

array(['regular gasoline', 'premium gasoline', 'diesel', 'natural gaz',
       'ethanol e85'], dtype=object)

In [10]:
fc_df.to_csv("../data/fuel_consumption_ratings.csv", index=False)

# Fuel Prices
The discription of this dataset is the following:
> This dataset contains weekly retail prices of three grades of gasoline, diesel, auto propane and compressed natural gas in 10 Ontario markets. Prices are in cents per litre (with compressed natural gas prices in cents per gasoline-equivalent litres)

We are only interested in the price of each year. To aggregate the values, we will choose the **median** price for all regions in a single week to represent the fuel price for that week, and then get the **median** price for all weeks in a year.

In [11]:
import numpy as nm

In [12]:
fp_df = pd.read_csv("../raw_data/fuel_prices/fueltypesall.csv")
fp_df.head()

,Date,Ottawa,Toronto West/Ouest,Toronto East/Est,Windsor,London,Peterborough,St. Catharine's,Sudbury,Sault Saint Marie,Thunder Bay,North Bay,Timmins,Kenora,Parry Sound,Ontario Average/Moyenne provinciale,Southern Average/Moyenne du sud de l'Ontario,Northern Average/Moyenne du nord de l'Ontario,Fuel Type,Type de carburant
0,1990-01-03,55.9,49.1,48.7,45.2,50.1,0.0,0.0,56.4,54.8,56.6,55.1,58.1,0.0,0.0,50.3,49.5,56.2,Regular Unleaded Gasoline,Essence sans plomb
1,1990-01-10,55.9,47.7,46.8,49.7,47.6,0.0,0.0,56.4,54.9,56.8,55.0,58.2,0.0,0.0,49.2,48.3,56.2,Regular Unleaded Gasoline,Essence sans plomb
2,1990-01-17,55.9,53.2,53.2,49.6,53.7,0.0,0.0,55.8,54.9,56.8,54.4,58.2,0.0,0.0,53.6,53.3,56.0,Regular Unleaded Gasoline,Essence sans plomb
3,1990-01-24,55.9,53.2,53.5,49.0,52.1,0.0,0.0,55.7,54.9,56.8,54.3,58.2,0.0,0.0,53.5,53.2,56.0,Regular Unleaded Gasoline,Essence sans plomb
4,1990-01-31,55.9,51.9,52.6,48.6,49.1,0.0,0.0,55.6,54.8,56.8,54.2,58.1,0.0,0.0,52.5,52.1,55.9,Regular Unleaded Gasoline,Essence sans plomb


the column `Type de carburant` is the french translation of the column `Fuel Type`, we will drop it.

In [13]:
fp_df.drop(labels="Type de carburant", axis="columns", inplace=True)

In [14]:
fp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10238 entries, 0 to 10237
Data columns (total 19 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Date                                           10238 non-null  object 
 1   Ottawa                                         10238 non-null  float64
 2   Toronto West/Ouest                             10238 non-null  float64
 3   Toronto East/Est                               10238 non-null  float64
 4   Windsor                                        10238 non-null  float64
 5   London                                         10238 non-null  float64
 6   Peterborough                                   10238 non-null  float64
 7   St. Catharine's                                10238 non-null  float64
 8   Sudbury                                        10238 non-null  float64
 9   Sault Saint Marie                              102

Generating median fuel price for each week

In [15]:
fp_df['weekly_price_median'] = fp_df.iloc[:, 1:18].aggregate("median", axis="columns", numeric_only=True)

In [16]:
fp_df.head()

,Date,Ottawa,Toronto West/Ouest,Toronto East/Est,Windsor,London,Peterborough,St. Catharine's,Sudbury,Sault Saint Marie,Thunder Bay,North Bay,Timmins,Kenora,Parry Sound,Ontario Average/Moyenne provinciale,Southern Average/Moyenne du sud de l'Ontario,Northern Average/Moyenne du nord de l'Ontario,Fuel Type,weekly_price_median
0,1990-01-03,55.9,49.1,48.7,45.2,50.1,0.0,0.0,56.4,54.8,56.6,55.1,58.1,0.0,0.0,50.3,49.5,56.2,Regular Unleaded Gasoline,50.1
1,1990-01-10,55.9,47.7,46.8,49.7,47.6,0.0,0.0,56.4,54.9,56.8,55.0,58.2,0.0,0.0,49.2,48.3,56.2,Regular Unleaded Gasoline,49.2
2,1990-01-17,55.9,53.2,53.2,49.6,53.7,0.0,0.0,55.8,54.9,56.8,54.4,58.2,0.0,0.0,53.6,53.3,56.0,Regular Unleaded Gasoline,53.6
3,1990-01-24,55.9,53.2,53.5,49.0,52.1,0.0,0.0,55.7,54.9,56.8,54.3,58.2,0.0,0.0,53.5,53.2,56.0,Regular Unleaded Gasoline,53.5
4,1990-01-31,55.9,51.9,52.6,48.6,49.1,0.0,0.0,55.6,54.8,56.8,54.2,58.1,0.0,0.0,52.5,52.1,55.9,Regular Unleaded Gasoline,52.5


In [17]:
fp_df['year'] = fp_df['Date'].map(lambda x: x[0:4]).astype(nm.int64)
fp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10238 entries, 0 to 10237
Data columns (total 21 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Date                                           10238 non-null  object 
 1   Ottawa                                         10238 non-null  float64
 2   Toronto West/Ouest                             10238 non-null  float64
 3   Toronto East/Est                               10238 non-null  float64
 4   Windsor                                        10238 non-null  float64
 5   London                                         10238 non-null  float64
 6   Peterborough                                   10238 non-null  float64
 7   St. Catharine's                                10238 non-null  float64
 8   Sudbury                                        10238 non-null  float64
 9   Sault Saint Marie                              102

Generate the median fuel price for each year, by fuel type

In [18]:
fp_df = fp_df.groupby(['year', 'Fuel Type']).agg({'weekly_price_median': 'median'})
fp_df.columns = ["price"]

In [19]:
fp_df.reset_index(inplace=True)

In [20]:
fp_df.head()

,year,Fuel Type,price
0,1990,Auto Propane,32.25
1,1990,Compressed Natural Gas,0.00
2,1990,Diesel,47.30
3,1990,Mid-Grade Gasoline,57.60
4,1990,Premium Gasoline,59.75


In [21]:
fp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       197 non-null    int64  
 1   Fuel Type  197 non-null    object 
 2   price      197 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.7+ KB


In [22]:
fp_df['Fuel Type'].unique()

array(['Auto Propane', 'Compressed Natural Gas', 'Diesel',
       'Mid-Grade Gasoline', 'Premium Gasoline',
       'Regular Unleaded Gasoline'], dtype=object)

Filter out the Fuel Types to only include "Compressed Natural Gas","Diesel","Premium Gasoline","Regular Unleaded Gasoline"

In [23]:
filter = fp_df['Fuel Type'].isin(["Compressed Natural Gas","Diesel","Premium Gasoline","Regular Unleaded Gasoline"])

In [24]:
fp_df[filter]['Fuel Type'].unique()

array(['Compressed Natural Gas', 'Diesel', 'Premium Gasoline',
       'Regular Unleaded Gasoline'], dtype=object)

In [25]:
fp_df = fp_df[filter]

In [26]:
fp_df_columns = ["year", "fuel_type", "price"]

In [27]:
fp_df.columns = fp_df_columns

In [28]:
fp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 1 to 196
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       131 non-null    int64  
 1   fuel_type  131 non-null    object 
 2   price      131 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.1+ KB


In [29]:
fp_df.to_csv("../data/fuel_prices.csv", index=False)